In [1]:
import numpy as np
import os

import plotly.graph_objs as go

from trProcess import *

In [7]:
tracenum = 10
baselinelist = {
    "FqCoDel": ["FqCoDel_0_5ms1000", "FqCoDel_0_500ms1000"], 
    "FIFO": ["PfifoFast_0_1000"],
    "DualQ": ["DualQ_9_1000"],
    # "CoDel": ["CoDel_0_5ms1000"],
    "Choke": ["Choke_0_1000"],
    "Hhf": ["Hhf_0_1000"],
    "Sfb": ["Sfb_0_1000"],
    "DWRR (Class, prelabeled)": ["Dwrr_9_1.00"],
    # "DWRR (Class, mislabeled)": ["Dwrr_19_1.00", "Dwrr_29_1.00", "Dwrr_39_1.00"],
    "Auto (Class, online)": ["Auto_0_1.00_ExpClass5.00"],
    "StrictiPriotity": ["StrictPriority_90_1000", "StrictPriority_9_1000"],
}

indir = "ns-3.34/logs/"

for exp in ["46_Video1_Web15040"
    # , "46_Video1_Web30040", "46_Video1_Web60040", "46_Video1_Web100040"
]:
    apps = exp.split("_")
    data1, data2 = [], []
    for name, baselines in baselinelist.items():
        p90Fct, maxAppA, maxAppB, jfi = [], [], [], []
        for baseline in baselines:
            p90FctTrace, maxAppATrace, maxAppBTrace, jfiTrace = [], [], [], []
            for traceid in range(tracenum):
                trace = "maxwell_30.0_0.01_300_%d.pitree-trace" % traceid
                conf = os.path.join (indir, exp, baseline, trace)
                    
                bws = []
                curMaxApp, curAppDur, _ = appTraceProcessor (os.path.join (conf, 'app.tr'))
                curMaxAppFlows = 0
                for port, appDelay in curMaxApp.items():
                    if port in range(10000, 20000):
                        curMaxAppFlows = max(curMaxAppFlows, appDelay)
                maxAppATrace.append(curMaxAppFlows)
     
                curAvgFct, curP90Fct, fctLastStart, fctFirstEnd, fcts = fctTraceProcessor(os.path.join (conf, 'fct.tr'))
                p90FctTrace.append(curP90Fct)
                bws = np.array(bws)
                jfiTrace.append(pow(np.nansum(bws),2) / (bws.size * np.nansum(np.square(bws))))
            p90Fct.append(np.average(p90FctTrace))
            maxAppA.append(np.average(maxAppATrace))
            jfi.append(np.average(jfiTrace))
            print(np.average(maxAppATrace), np.average(p90FctTrace), baseline)
                
        data1.append(go.Scatter(
            x=maxAppA,
            y=p90Fct,
            mode="markers+text",
            text=baselines,
            name=name,
            textposition="bottom center"
        ))
    fig = go.Figure(data=data1)
    fig.update_layout(
        title=exp,
        xaxis_title="Video Latency Jitter (ms)",
        yaxis_title="Page Loading Time (ms)"
    )
    fig.show()

/tmp/ipykernel_32412/3504602975.py:42: RuntimeWarning:

invalid value encountered in double_scalars



185.3 889.1299999999999 FqCoDel_0_5ms1000
199.1 187.9 FqCoDel_0_500ms1000
187.5 212.61999999999998 PfifoFast_0_1000
146.0 1230.1299999999999 DualQ_9_1000
552.1 1272.69 Choke_0_1000
110.6 98.08000000000001 Hhf_0_1000
5185.4 209.82 Sfb_0_1000
83.3 311.12 Dwrr_9_1.00
102.5 268.81 Auto_0_1.00_ExpClass5.00
199.3 184.41 StrictPriority_90_1000
79.9 401.52 StrictPriority_9_1000
